Descripción general del sistema de Deep Learning

El sistema está diseñado para realizar reducción de ruido en audio mediante técnicas de deep learning. La entrada del modelo corresponde al espectro de magnitudes de la señal de audio con ruido, mientras que la salida objetivo corresponde al espectro de magnitudes de la señal limpia. La red neuronal toma como entrada el espectro de magnitudes con ruido y estima el espectro de magnitudes de la señal denoised.
El entrenamiento se realiza minimizando el error cuadrático medio (MSE) entre el espectro predicho y el espectro objetivo. La arquitectura de la red está compuesta por capas totalmente conectadas que procesan segmentos de la señal en el dominio frecuencia–tiempo.

Tras la inferencia, la señal denoised se reconstruye en el dominio del tiempo combinando el espectro de magnitudes predicho con la información de fase original de la señal con ruido. Esta reconstrucción se lleva a cabo mediante la transformada rápida de Fourier inversa de ventana corta (ISTFT). El resultado es una forma de onda con el ruido significativamente reducido, y que se aproxima a la señal limpia de referencia.

In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

import Utils.model_setup

In [ ]:
window_size = 256
model_params= {"window_length": window_size,
"overlap": int(0.75 * window_size),
"fft_length": window_size,
"num_features": window_size // 2 + 1,
"num_segments": 8,
"batch_size": 128,
"epochs": 3,
"lr": 1e-5}

clean_folder = "../Audios/Dataset/dev-clean"

noisy_folder = "../Audios/Augmented Audios 3 SNR"

noisy_segments_list, clean_segments_list = Utils.model_setup.initialize(model_params, clean_folder, noisy_folder)

X, y = Utils.model_setup.batch_preprocessing(clean_segments_list, noisy_segments_list)

torch.Size([160586, 1, 129, 8])

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
import random
import numpy as np
from pathlib import Path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_samples = X.shape[0]
indices = torch.randperm(num_samples)
train_len = int(0.99 * num_samples)
train_idx = indices[:train_len]
val_idx = indices[train_len:]
lr=1e-5
batch_size=128
num_segments=8
num_features=129

train_dataset = torch.utils.data.TensorDataset(X[train_idx], y[train_idx])
val_dataset = torch.utils.data.TensorDataset(X[val_idx], y[val_idx])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

model = Utils.model_setup.DenoiseNet(num_features, num_segments).to(device)

# ======================
# 7. Entrenamiento
# ======================
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [4]:
epochs=3

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)

    # Validación
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            loss = criterion(preds, yb)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_loss:.6f} - Val Loss: {avg_val_loss:.6f}")

Epoch 1/3 - Train Loss: 0.316396 - Val Loss: 0.132619
Epoch 2/3 - Train Loss: 0.136404 - Val Loss: 0.084592
Epoch 3/3 - Train Loss: 0.106992 - Val Loss: 0.069854


In [ ]:

torch.save(model, "denoise_model_complete.pth")

NameError: name 'dataset' is not defined